### Requisite Imports

In [1]:
import getpass
username=input('username = ')
password = getpass.getpass('password = ')

username =  nijesh
password =  ········


### Define the WML API Credentials for a diff cluster

In [2]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import warnings
warnings.filterwarnings("ignore")
import os


# #anup
wml_credentials = {
    "url": "https://dhl-cpd-dhl.apps.anup-dal12-demo-cpdga.demo.ibmcloud.com",
    "username": username,
    "password": password,
   "instance_id" : "openshift",
   "version": "2.5.0"
    
}


client = WatsonMachineLearningAPIClient(wml_credentials)

### Query Spaces

In [3]:
client.spaces.list(limit=5)

------------------------------------  -------------------  ------------------------
GUID                                  NAME                 CREATED
df23f969-8d4d-46a2-beb6-e248d3c44c18  DHL_Production       2020-05-18T04:18:40.949Z
c3e7d48b-0acf-4f95-8cf5-e414ebaa46fd  openscale-fast-path  2020-05-11T04:04:25.714Z
513d9c2a-ddb1-4055-8f8d-09da56d7ac40  DHL_Staging          2020-05-01T11:23:33.934Z
4c267a48-24d0-420b-a1b3-f9e819713be9  CP4DTEST             2020-03-31T08:50:10.470Z
------------------------------------  -------------------  ------------------------


### Fetch GUID

In [4]:
def guid_from_space_name(client, space_name):
    instance_details = client.service_instance.get_details()
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

In [11]:
space_uid = guid_from_space_name(client,"DHL_Production")
space_uid

'df23f969-8d4d-46a2-beb6-e248d3c44c18'

### Set default Space

In [12]:
client.set.default_space(space_uid)

'SUCCESS'

### Promote Data Assset to Deployment space

In [7]:
client.data_assets.create("eval_mdd.tsv","/project_data/data_asset/eval_mdd.tsv")
client.data_assets.create("preprocess.py","/home/wsuser/work/project_git_repo/dhl_ibm_cp4d/assets/jupyterlab/utils.py")
client.data_assets.create("config.py","/home/wsuser/work/project_git_repo/dhl_ibm_cp4d/assets/jupyterlab/config.py")

Creating data asset...
SUCCESS
Creating data asset...
SUCCESS
Creating data asset...
SUCCESS


{'metadata': {'space_id': 'df23f969-8d4d-46a2-beb6-e248d3c44c18',
  'guid': '0638d2d3-6e0d-4d8d-a8f5-bb9b7ff4bf44',
  'href': '/v2/data_assets/0638d2d3-6e0d-4d8d-a8f5-bb9b7ff4bf44?space_id=df23f969-8d4d-46a2-beb6-e248d3c44c18',
  'asset_type': 'data_asset',
  'created_at': '2020-05-18T04:22:40Z',
  'last_updated_at': '2020-05-18T04:22:40Z'},
 'entity': {'data_asset': {'mime_type': 'text/x-python', 'dataset': False}}}

### Promote Latest Model to Deployment space

In [13]:
#latest_model_file_name
#client.data_assets.create("MDD_v2.pkl","/project_data/data_asset/MDD_v2.pkl")

In [10]:
client.data_assets.list()

-------------  ----------  -------  ------------------------------------
NAME           ASSET_TYPE  SIZE     ASSET_ID
preprocess.py  data_asset  6977     13492590-8a8c-4139-baa1-21c8005aa7fb
eval_mdd.tsv   data_asset  300542   fdf35562-2cba-4987-9421-0fa2b0e15a38
config.py      data_asset  11043    1222f0b2-d93b-4554-8e4e-caa1301736d1
MDD_v2.model   data_asset  102802   85025165-1810-47e5-9a0a-5b171c549d2a
MDD_v1.model   data_asset  4322377  05b2523a-3548-45cd-ab48-00052d7d5de9
-------------  ----------  -------  ------------------------------------


### Initialize the deployment function parameters

In [13]:
model_details = client.data_assets.get_details("db59d720-176b-4919-87fe-f858b138c269") #v2(Preprod)
#model_details = client.data_assets.get_details("05b2523a-3548-45cd-ab48-00052d7d5de9") #v1(Challenger)
model_id = client.data_assets.get_uid(model_details)
model_href = client.data_assets.get_href(model_details)


data_details = client.data_assets.get_details("ae9ca351-38cb-402a-b04c-18950173c78c")
data_id = client.data_assets.get_uid(data_details)


script_details = client.data_assets.get_details("f25d1847-e2b6-4bc6-bd7a-a752e95486c8")
script_id = client.data_assets.get_uid(script_details)

config_script_details = client.data_assets.get_details("0638d2d3-6e0d-4d8d-a8f5-bb9b7ff4bf44")
config_script_id = client.data_assets.get_uid(config_script_details)

project_uid = os.environ['PROJECT_ID']

params = {
    "wml_credentials": wml_credentials,
    "project_uid": project_uid,
    "space_uid": space_uid,
    "mdd_model_id": model_id,
    "data_details":data_id,
    "preprocess_scripts":script_id,
    "config_scripts":config_script_id
    
    
}


### Deployable MDD Python Closure Function

In [14]:
def mdd_deployment_function(params=params):
    import subprocess
    import sys,time
    import pickle
    from watson_machine_learning_client import WatsonMachineLearningAPIClient
    
    params['wml_credentials']['instance_id'] = 'openshift'
    
    sys.path.insert(0,'/home/wmlfuser/.local/lib/python3.6/site-packages/')
    sys.path.insert(0, './')

    
    # Setup Client
    client = WatsonMachineLearningAPIClient(params['wml_credentials'])
    client.set.default_space(params['space_uid'])
    
    #Download Required Models and Files locally
    client.data_assets.download(params['preprocess_scripts'],"preprocess.py")
    client.data_assets.download(params['config_scripts'],"config.py")
    client.data_assets.download(params['mdd_model_id'],"mdd.model")
    #pro_data = pd.read_csv('/project_data/data_asset/eval_mdd.tsv', sep='\t', na_values='?')
    
    
    #Payload Preprocessing 
    
    def payload_process(data):
        import preprocess
        import config
        import pandas as pd
        import numpy as np
        
#         data = pd.DataFrame(values,columns=fields)
#         data.replace('OPENSCALE','',inplace=True)
#         data.replace(99999,np.nan,inplace = True)
            
        #Preprocessing of the Evaluation Input

        data = data.drop(['weight'], axis=1, errors='ignore')

        # ----- Some intial preprocessing calculations -----
        data.new_potential_revenue.replace(0, np.nan, inplace = True)
        data.published_revenue.replace(0, np.nan, inplace = True)
        data.shipments.replace(0, np.nan, inplace = True)
        data.wps.replace(0, np.nan, inplace = True)
        data.insert(11,'weight',data['wps']*data['shipments'])
        data[['weight',
                'published_revenue',
                'new_potential_revenue']] = data[['weight','published_revenue',
                                            'new_potential_revenue']].astype(float)
        data.rename(columns={
                    'reason_for_lead'                :'REASONFORLEAD'
                    ,'lead_source_type'                   :'LEADSOURCE'
                    ,'industry_code'                     :'INDUSTRY'
                    ,'primary_competitor'                     :'MAINCOMP'
                    }, inplace=True)

        label_col = "discount"
        bin_col = ['ecomm']
        cat_col = ['industry','product_cluster','loyalty_code','opportunity_type','reason_for_lead','lead_source_type','competitor']

        num_col = [
        'log_published_revenue', # log(published_revenue)
        'log_published_revenue_sq', # (log(published_revenue))^2
        'log_potential_revenue', # log(new_potential_revenue)
        'log_offer_published_revenue', # log(offer_published_revenue)
        'log_shipments', # log(shipments)
        'log_wps', # log(wps)
        'log_weight', # log(shipments x wps)
        'PROD.MIX:DDEXPORT',
        'PROD.MIX:DDIMPORT',
        'PROD.MIX:TD3RD',
        'PROD.MIX:DOMESTIC',
        'PROD.MIX:TDEXPORT',
        'PROD.MIX:TDIMPORT']

        slice = data.loc[(data['organization_id'] == data['organization_id'].unique()[0])]
        slice['physical_channel'] = slice.physical_channel.astype('str').replace(".",'')
        slice = preprocess.preprocess(slice,label_col,bin_col,num_col,cat_col)
        
        return slice
 

    def score(payload):
        
        import preprocess
        import config
        import pandas as pd
        import numpy as np
        import pickle
        
        field_values =[]
              
        try:
            #Load the Saved Model
            
            with open('mdd.model','rb') as f:
                final_mdd_model = pickle.load(f)
                
            #Decode a Dataframe from Payload Json

            values = payload['input_data'][0]['values']
            fields = payload['input_data'][0]['fields']
            
            data = pd.DataFrame(values,columns=fields)
            data.replace('OPENSCALE','',inplace=True)
            data.replace(99999,0,inplace = True)
            
            categorical_columns = ['industry_code','product_cluster','reason_for_lead','lead_source_type','primary_competitor','contactrole','physical_channel']
            feature_columns = ['offer_id','organization_id','recommended_revenue','recommended_pid','recommended_pid_name','physical_channel','published_revenue','new_potential_revenue','shipments','wps', 'product_cluster', 'industry_code','contactrole','reason_for_lead', 'lead_source_type','primary_competitor']
            non_numerical = ['recommended_pid_name','offer_id','organization_id'] + categorical_columns
            numerical = [each for each in feature_columns if each not in non_numerical]
            for each in numerical:
                if not each =='shipments':
                    data[each] = data[each].astype('float64')
                else:
                    data[each] = data[each].astype('int')
                    
            for each in non_numerical:
                data[each] = data[each].astype('str')
                
            #data['recommended_pid'] = data['recommended_pid'].astype('str')
                       
            values = data.values.tolist()
            
            #Preprocess the raw data load before scoring
            slice = payload_process(data)
            
            
            X = slice
            y = final_mdd_model.predict(slice)
            
            #return {'predictions':[{'results': list(y)}]}
            
        
           
            y = list(y)
            
            i = 0
            while i < len(y) :
                values[i].append(y[i])
                i +=1
                    
            return {'predictions':[{'fields':fields+['predictions'],'values': values}]}
        
        except Exception as e:
            return {"predictions": [{"error" : repr(e)}]}
        
    return score




In [15]:
# results = mdd_deployment_function()(payload_data)
# results

### Store the Function in deployment space

In [16]:
PRE_PROD_MODEL_NAME="DHL MDD Model-PreProd"
PRE_PROD_DEPLOYMENT_NAME="DHL MDD Model-PreProd"

PRE_PROD_CHALLENGER_MODEL_NAME="DHL MDD Model-Challenger"
PRE_PROD_CHALLENGER_DEPLOYMENT_NAME="DHL MDD Model-Challenger"

PRE_PROD_SPACE_NAME="DHL_Staging"

In [17]:
client.set.default_space(space_uid)

'SUCCESS'

### Function Metadata.

In [18]:
meta_props = {
    client.repository.FunctionMetaNames.NAME: 'DHL MDD Model-Challenger',
    client.repository.FunctionMetaNames.RUNTIME_UID: "ai-function_0.1-py3.6",
    client.repository.FunctionMetaNames.SPACE_UID: space_uid
    
}

# meta_props = {
#     client.repository.FunctionMetaNames.NAME: 'DHL MDD Model-PreProd',
#     client.repository.FunctionMetaNames.RUNTIME_UID: "ai-function_0.1-py3.6",
#     client.repository.FunctionMetaNames.SPACE_UID: space_uid
# }

### Create the Function artifact.

In [19]:
function_artifact = client.repository.store_function(meta_props=meta_props, function=mdd_deployment_function)
function_uid = client.repository.get_function_uid(function_artifact)
print("Function UID = " + function_uid)

Function UID = 480a693a-1f17-49c2-bddc-eea75d2770b7


### Details about the function.

In [20]:
function_details = client.repository.get_details(function_uid)
from pprint import pprint
pprint(function_details)

{'entity': {'name': 'DHL MDD Model-Challenger',
            'runtime': {'href': '/v4/runtimes/ai-function_0.1-py3.6'},
            'space': {'href': '/v4/spaces/df23f969-8d4d-46a2-beb6-e248d3c44c18'},
            'type': 'python'},
 'metadata': {'created_at': '2020-05-20T05:04:35.002Z',
              'guid': '480a693a-1f17-49c2-bddc-eea75d2770b7',
              'href': '/v4/functions/480a693a-1f17-49c2-bddc-eea75d2770b7?space_id=df23f969-8d4d-46a2-beb6-e248d3c44c18',
              'id': '480a693a-1f17-49c2-bddc-eea75d2770b7',
              'modified_at': '2020-05-20T05:04:37.002Z',
              'owner': '1000331001'}}


### Deploy the MDD Function

In [21]:
# # Deployment metadata.
# deploy_meta = {
#     client.deployments.ConfigurationMetaNames.NAME: "DHL MDD Model-PreProd",
#     client.deployments.ConfigurationMetaNames.DESCRIPTION: "mdd_deployment_function-deploy",
#     client.deployments.ConfigurationMetaNames.ONLINE: {}
# }

#Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "DHL MDD Model-Challenger",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "mdd_deployment_function-deploy",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

### Create the deployment.

In [22]:
deployment_details = client.deployments.create(function_uid, meta_props=deploy_meta)
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment uid = {}'.format(deployment_uid))



#######################################################################################

Synchronous deployment creation for uid: '480a693a-1f17-49c2-bddc-eea75d2770b7' started

#######################################################################################


initializing...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='41f17acb-078e-4754-b90f-1eac2196e400'
------------------------------------------------------------------------------------------------


Deployment uid = 41f17acb-078e-4754-b90f-1eac2196e400


### Prepare scoring payload.

In [23]:
from cpd_utils import *
payload_data = prepare_scoring_payload(path='/project_data/data_asset/eval_mdd.tsv',no_of_records=1)

fields = payload_data['input_data'][0]['fields']
values = payload_data['input_data'][0]['values']

job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': fields,
        'values': values
    }]
}

### MDD Model Scoring

In [24]:
job_details = client.deployments.score(deployment_uid, job_payload)
job_details

{'predictions': [{'fields': ['offer_id',
    'organization_id',
    'recommended_revenue',
    'recommended_pid',
    'recommended_pid_name',
    'physical_channel',
    'published_revenue',
    'new_potential_revenue',
    'shipments',
    'wps',
    'product_cluster',
    'industry_code',
    'contactrole',
    'reason_for_lead',
    'lead_source_type',
    'primary_competitor',
    'predictions'],
   'values': [['CL_O_115157344685',
     'DHL CL',
     21.442411764705888,
     nan,
     'TDI_36%',
     'nan',
     31.4364,
     24.0,
     2,
     5.0,
     'TDImport',
     'TECHCSEL',
     'Influencer',
     'Cash Customer',
     'CSV',
     'Other',
     36.91025922516022]]}]}